# Exploring the LSTM model for stock price prediction

### Load the data


In [1]:
import pandas as pd
import datetime
from tensorflow import keras
import data_engineering
import simulation
import numpy as np
import lstm_model

In [2]:
# load data
df_dict = data_engineering.separate_by_stock()

df_dict = data_engineering.fillna(df_dict)

cv_trades = [{}, {}, {}, {}]
cv_opens = [{}, {}, {}, {}]
for tick in df_dict:
    train, test = data_engineering.train_test_split(df_dict[tick])
    
    features = ["finvader_tot", "pos_art_count", "total_articles", "Open_Diff", "y", "Open"]

    train, test = train[features], test[features]

    #accs = np.zeros(4)
    i = 0
    for train_idx, test_idx in data_engineering.get_cv_splits(train):
        cv_opens[i][tick] = train.loc[test_idx, "Open"]

        df_tt = train.loc[train_idx].drop(columns=['Open'])
        df_ho = train.loc[test_idx].drop(columns=['Open'])

        pred_change, trades = lstm_model.run_lstm_model(df_tt, df_ho)
        cv_trades[i][tick] = trades
        i+=1
    
for i in range(len(trades)):
    print(simulation.get_performance(cv_trades[i], cv_opens[i]))


746/746 [==============================] - 1s 2ms/step - loss: 0.0358
Test RMSE: 3.596
810/810 [==============================] - 2s 2ms/step - loss: 0.0402
Test RMSE: 2.951
874/874 [==============================] - 3s 3ms/step - loss: 0.0416
Test RMSE: 3.509
937/937 [==============================] - 2s 2ms/step - loss: 0.0448
Test RMSE: 2.472
746/746 [==============================] - 3s 3ms/step - loss: 0.0523
Test RMSE: 2.424
810/810 [==============================] - 3s 4ms/step - loss: 0.0474
Test RMSE: 1.983
874/874 [==============================] - 2s 2ms/step - loss: 0.0500
Test RMSE: 2.332
937/937 [==============================] - 4s 4ms/step - loss: 0.0497
Test RMSE: 1.610
746/746 [==============================] - 2s 3ms/step - loss: 0.0418
Test RMSE: 4.234
810/810 [==============================] - 2s 2ms/step - loss: 0.0461
Test RMSE: 3.687
874/874 [==============================] - 2s 2ms/step - loss: 0.0476
Test RMSE: 3.885
937/937 [==============================] - 

KeyError: 0

In [26]:
def get_performance(trade_dict, test_dict):
    n = len(trade_dict["AAPL"])
    x_t = [1] * n
    for i in range(1,n):
        x_t[i] = x_t[i-1] / 2
        for tick in trade_dict:
            x_t[i] += (x_t[i-1] / 30) * (1 + trade_dict[tick][i-1] * (test_dict[tick][i] - test_dict[tick][i-1]) / test_dict[tick][i-1])
    return x_t[-1]


In [29]:
for i in range(4):
    print(get_performance(cv_trades[i], cv_opens[i]))

1.0017263591687249
1.0187007416410196
1.0244900681192592
0.9974660101393773


In [25]:
samp_trade = np.array([1,1])
samp_open = np.array([1,2])
samp_t_dict = {tick:samp_trade.copy() for tick in df_dict}
samp_o_dict = {tick:samp_open.copy() for tick in df_dict}

print(get_performance(samp_t_dict, samp_o_dict))

samp_trade = np.array([-1,1])
samp_open = np.array([1,2])
samp_t_dict = {tick:samp_trade.copy() for tick in df_dict}
samp_o_dict = {tick:samp_open.copy() for tick in df_dict}

print(get_performance(samp_t_dict, samp_o_dict))



0.5
0.5666666666666667
0.6333333333333333
0.7
0.7666666666666666
0.8333333333333333
0.8999999999999999
0.9666666666666666
1.0333333333333332
1.0999999999999999
1.1666666666666665
1.2333333333333332
1.2999999999999998
1.3666666666666665
1.4333333333333331
1.4999999999999998
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5


In [30]:
cv_trades = [{}, {}, {}, {}]
cv_opens = [{}, {}, {}, {}]
for tick in df_dict:
    train, test = data_engineering.train_test_split(df_dict[tick])
    
    features = ["finvader_tot", "Open_Diff", "y", "Open"]

    train, test = train[features], test[features]

    #accs = np.zeros(4)
    i = 0
    for train_idx, test_idx in data_engineering.get_cv_splits(train):
        cv_opens[i][tick] = train.loc[test_idx, "Open"]

        df_tt = train.loc[train_idx].drop(columns=['Open'])
        df_ho = train.loc[test_idx].drop(columns=['Open'])

        pred_change, trades = lstm_model.run_lstm_model(df_tt, df_ho, epochs=4)
        cv_trades[i][tick] = trades
        i+=1
    
for i in range(len(trades)):
    print(simulation.get_performance(cv_trades[i], cv_opens[i]))

746/746 [==============================] - 1s 2ms/step - loss: 0.0359
Test RMSE: 3.625
810/810 [==============================] - 2s 2ms/step - loss: 0.0403
Test RMSE: 2.935
874/874 [==============================] - 1s 2ms/step - loss: 0.0416
Test RMSE: 3.504
937/937 [==============================] - 3s 4ms/step - loss: 0.0445
Test RMSE: 2.466
746/746 [==============================] - 1s 2ms/step - loss: 0.0535
Test RMSE: 2.419
810/810 [==============================] - 2s 2ms/step - loss: 0.0471
Test RMSE: 1.973
874/874 [==============================] - 5s 6ms/step - loss: 0.0509
Test RMSE: 2.373
937/937 [==============================] - 4s 4ms/step - loss: 0.0499
Test RMSE: 1.600
746/746 [==============================] - 4s 5ms/step - loss: 0.0420
Test RMSE: 4.226
810/810 [==============================] - 2s 2ms/step - loss: 0.0459
Test RMSE: 3.712
874/874 [==============================] - 3s 3ms/step - loss: 0.0479
Test RMSE: 3.821
937/937 [==============================] - 

KeyError: 0

In [31]:
for i in range(4):
    print(get_performance(cv_trades[i], cv_opens[i]))

0.9814018520046874
1.0065043428058786
1.0216227406602079
0.9892340910095337


In [49]:

train, test = data_engineering.train_test_split(df_dict['AAPL'])
train, test = train[features], test[features]


## Scales the data

In [11]:
from sklearn.preprocessing import MinMaxScaler

X, y = train.drop(columns=['y']).values, train.y.values

X_test, y_test = test.drop(columns=['y']).values, test.y.values

scaler_X = MinMaxScaler(feature_range=(-1,1))
scaler_y = MinMaxScaler(feature_range=(-1,1))

# scales the open_diff column to be between -1 and 1
X[:,1:] = scaler_X.fit_transform(X[:,1:])
y = scaler_y.fit_transform(y.reshape(-1,1)).reshape(-1,)

X_test[:,1:] = scaler_X.fit_transform(X_test[:,1:])
y_test = scaler_y.transform(y_test.reshape(-1,1)).reshape(-1,)



## Fits the model using scaled data

## Creating the LSTM model

In [12]:
def fit_lstm(X, y, batch_size, nb_epoch, neurons):
    X = X.reshape(X.shape[0], 1, X.shape[1])
    model = keras.Sequential()
    model.add(keras.layers.LSTM(neurons, batch_input_shape=(1, X.shape[1], X.shape[2]), stateful=True))
    model.add(keras.layers.Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    for i in range(nb_epoch):
        model.fit(X, y, epochs=1, batch_size=batch_size, verbose=1, shuffle=False)
        model.reset_states()
    return model

In [14]:
model = fit_lstm(X, y, 1, 10, 3)

997/997 [==============================] - 1s 1ms/step - loss: 0.0451


In [15]:
def forecast_lstm(model, batch_size, X):
    X = X.reshape(-1, 1, X.shape[1])
    yhat = model.predict(X, batch_size=batch_size)
    return yhat[0,0]

In [17]:
forecast_lstm(model, 1, X_test)

0.11813852

In [44]:
from pandas import DataFrame
from pandas import Series
from pandas import concat
from pandas import read_csv
from pandas import datetime
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from math import sqrt
#from matplotlib import pyplot
import numpy as np
 
# date-time parsing function for loading the dataset
def parser(x):
    return datetime.strptime('190'+x, '%Y-%m')
 
# invert differenced value
def inverse_difference(history, yhat, interval=1):
    return yhat + history[-interval]
 
# scale train and test data to [-1, 1]
def scale(train, test):
    # fit scaler
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(train)
    # transform train
    train = train.reshape(train.shape[0], train.shape[1])
    train_scaled = scaler.transform(train)
    # transform test
    test = test.reshape(test.shape[0], test.shape[1])
    test_scaled = scaler.transform(test)
    return scaler, train_scaled, test_scaled
 
# inverse scaling for a forecasted value
def invert_scale(scaler, X, value):
    new_row = [x for x in X] + [value]
    array = numpy.array(new_row)
    array = array.reshape(1, len(array))
    inverted = scaler.inverse_transform(array)
    return inverted[0, -1]
 
# fit an LSTM network to training data
def fit_lstm(X, y, batch_size, nb_epoch, neurons):
    X = X.reshape(X.shape[0], 1, X.shape[1])
    model = keras.Sequential()
    model.add(keras.layers.LSTM(neurons, batch_input_shape=(1, X.shape[1], X.shape[2]), stateful=True))
    model.add(keras.layers.Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    for i in range(nb_epoch):
        model.fit(X, y, epochs=1, batch_size=batch_size, verbose=1, shuffle=False)
        model.reset_states()
    return model
 
# make a one-step forecast
def forecast_lstm(model, batch_size, X):
    X = X.reshape(1, 1, len(X))
    yhat = model.predict(X, batch_size=batch_size)
    return yhat[0,0]


X, y = train.drop(columns=['y']).values, train.y.values

X_test, y_test = test.drop(columns=['y']).values, test.y.values

scaler_X = MinMaxScaler(feature_range=(-1,1))
scaler_y = MinMaxScaler(feature_range=(-1,1))

# scales the open_diff column to be between -1 and 1
X[:,1:] = scaler_X.fit_transform(X[:,1:])
y = scaler_y.fit_transform(y.reshape(-1,1)).reshape(-1,)

X_test[:,1:] = scaler_X.fit_transform(X_test[:,1:])




model = fit_lstm(X, y, 1, 10, 3)



# forecast the entire training dataset to build up state for forecasting
X_reshaped = X.reshape(X.shape[0], 1, X.shape[1])
model.predict(X_reshaped, batch_size=1)
 
# walk-forward validation on the test data
predictions = list()
for i in range(X_test.shape[0]):
    # make one-step forecast
    yhat = forecast_lstm(model, 1, X_test[i,:])
    # invert scaling
    yhat = scaler_y.inverse_transform(np.array([yhat]).reshape(1,1))[0,0]
    
    # invert differencing
    #pred_open = open_prices.iloc[i] + yhat
    # store forecast
    predictions.append(yhat)
    
    actual = y_test[i]
    print('Day=%d, Predicted=%f, Expected=%f' % (i+1, yhat, actual))

print(len(predictions)==len(y_test))
# report performance
rmse = sqrt(mean_squared_error(y_test, predictions))
print('Test RMSE: %.3f' % rmse)
# line plot of observed vs predicted
#pyplot.plot(raw_values[-12:])
#pyplot.plot(predictions)
#pyplot.show()

C:\Users\timal\AppData\Local\Temp\ipykernel_17704\1953027932.py:5: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  from pandas import datetime


997/997 [==============================] - 1s 1ms/step - loss: 0.0446
Day=1, Predicted=0.007017, Expected=-2.433677
Day=2, Predicted=0.105804, Expected=3.635619
Day=3, Predicted=-0.272396, Expected=5.711717
Day=4, Predicted=-0.273073, Expected=-0.089395
Day=5, Predicted=-0.128553, Expected=-0.884074
Day=6, Predicted=-0.137814, Expected=0.745013
Day=7, Predicted=-0.263830, Expected=-3.327703
Day=8, Predicted=-0.089437, Expected=-2.384027
Day=9, Predicted=-0.160631, Expected=3.446900
Day=10, Predicted=-0.372534, Expected=-0.089405
Day=11, Predicted=-0.266800, Expected=0.963540
Day=12, Predicted=-0.232495, Expected=3.893906
Day=13, Predicted=-0.601035, Expected=-1.003256
Day=14, Predicted=-0.466746, Expected=2.235010
Day=15, Predicted=-0.546822, Expected=1.966822
Day=16, Predicted=-0.577329, Expected=-0.466890
Day=17, Predicted=-0.279457, Expected=0.029827
Day=18, Predicted=-0.296565, Expected=1.072806
Day=19, Predicted=-0.259099, Expected=-1.956885
Day=20, Predicted=-0.187863, Expected=1

In [45]:
pred_rise_fall = predictions / np.abs(predictions)
actual_rise_fall = y_test / np.abs(y_test)

In [46]:
from sklearn.metrics import accuracy_score
accuracy_score(pred_rise_fall, actual_rise_fall)

0.46863468634686345

In [5]:
import lstm_model

In [12]:
pred_change, trades = lstm_model.run_lstm_model(train, test)

997/997 [==============================] - 1s 1ms/step - loss: 0.0437
Test RMSE: 2.391


In [14]:
import numpy as np
from sklearn.metrics import accuracy_score

In [15]:
accuracy_score(test.y / np.abs(test.y), trades)

0.47232472324723246

NameError: name 'np' is not defined